In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import panda as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("chemin vers la matrice", index_col=0)
X=
Y=

In [ ]:
X_log = np.log2(X + 1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(Y)
n_classes = len(label_encoder.classes_)

In [ ]:
X_selection=#selon les citères qu'on veut, on peut prendre les HVG
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selection)

# Split train/validation/test
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"\nTailles des ensembles:")
print(f"Train: {X_train.shape[0]}")
print(f"Validation: {X_val.shape[0]}")
print(f"Test: {X_test.shape[0]}")


Modèle dense à 4 couches

In [ ]:


model = keras.Sequential([
    
    layers.Input(shape=(input_dim,)),
    layers.BatchNormalization(),
    
   
    layers.Dense(512, activation='relu', 
                kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    

    layers.Dense(256, activation='relu',
                kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    
    layers.Dense(128, activation='relu',
                kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    

    layers.Dense(64, activation='relu',
                kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dropout(0.3),
    
  
  
    layers.Dense(n_classes, activation='softmax')
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()


In [ ]:
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)

In [ ]:
#Faire une matrice de confusion